In [1]:
import numpy as np
import datetime as dt
import pandas as pd

In [2]:
df = pd.read_excel("/Users/administrador/Downloads/novo_excel_HOBO_UNIFICADA_27.10.2022.xlsx")

In [3]:
#formatando nome de coluna
df.rename(columns={'Sítio': 'sitio', 'Temp (ºC)': 'temp', 'Int Lum (lux)': 'int_lum_lux',
                'Unidade Hobo': 'unidade_hobo','Hora': 'hora', 'formatted': 'data'}, inplace = True)

In [4]:
#tirando a colunas desnecessárias
df.drop(columns = ['Unnamed: 0', 'Data', 'Mês', 'Ano'], inplace = True)

In [5]:
#criando uma copia do df para comparar os métodos
df2 = df

In [6]:
%%timeit
#criando funcão para identificar o tipo de dado de hora aplicar a substituicao correta
def corrigir_hora(registro):
    
    ''''DESCRIÇÃO: como são vários tipos de dados e formatos dentro da coluna hora, serão feitos testes para
    verificar o tipo de dado da linha e aplicar as transformacões adequadas para que tudo seja padronizado'''
    
    #primeiro caso: int ou float. Significa que há problema no dado, então será colocado um marcador pra isso
    if type(registro) == int or type(registro) == float:
         return np.nan
    
    #segundo caso: string
    elif type(registro) == str:
        #como alguns horarios menores que 10 que não possuem o zero no início (e.g: 7:56) o zfill corrige isso
        registro.zfill(5)
        
        #adicionar os segundos para as linhas que não os possuem
        if len(registro) < 6:
            registro = registro + ':00'
        else:
            pass
        
        #convertendo enfim para time
        return pd.to_datetime(registro, format='%H:%M:%S').time()
        
    #terceiro caso: datetime. da para converter direto para time (por algum motivo precisa do pd.to_datetime)
    else:
        return pd.to_datetime(registro, format='%H:%M:%S').time()
    
#aplicando a função corrigir_hora
df.loc[:, 'hora_corrigida'] = df.loc[:, 'hora'].map(corrigir_hora)

#convertando data de volta a date para que a próxima parte possa funcionar
df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d').dt.date

#converter tudo de volta pra string nao parece o jeito mais prático, mas é o que tem pra agora
df.loc[:, 'datetime'] = np.where(df.hora_corrigida != df.hora_corrigida,
                                     np.nan, df['data'].astype(str) + ' ' + df['hora_corrigida'].astype(str))

#por fim colocando tudo de volta pra datetime
df.loc[:, 'datetime'] = pd.to_datetime(df.loc[:, 'datetime'], format='%Y-%m-%d %H:%M:%S')

1min 16s ± 1.85 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
#Definindo as funções
def numerico(registro):
    
    '''foi visto antes que valores numéricos para coluna de hora não são válidos'''
    
    if type(registro) == int or type(registro) == float:
        return '1900-01-01 00:00:00' #designei uma string para indicar erro, ano 1900!!
    else:
        return registro
    
def normalize_str(registro):
    
    '''Padronizando o tamanho das strings'''
    
    #colocando os segundos quando estes estão ausentes
    if len(registro) < 6:
        return registro + ':00'
    
    #quando a linha na coluna de hora está com data
    elif len(registro) > 15:
        return registro[11:] #cortando fora a parte de date e ficando com time
    
    else:
        return registro

#Chamando função para lidar com dados numericos
df2.loc[:, 'hora'] = df2.loc[:, 'hora'].map(numerico)

#convertando todo mundo para str
df2.hora = df2.hora.astype(str)

#zero filling- padronizando a hora pra ficar sempre com 2 algarismos (e.g. 7:50 > 07:50)
df2.hora.str.zfill(5)

#Chamando função para padronizar as strings
df2.loc[:, 'hora'] = df2.loc[:, 'hora'].map(normalize_str)

#algumas linhas de data tem o horario tbm -> ficando então só com a data
df2['data'] = pd.to_datetime(df2['data'], format='%Y-%m-%d').dt.date

#Formando a coluna datetime juntando date com time
df2.loc[:, 'datetime'] = df2['data'].astype(str) + ' ' + df2['hora']

#Convertando para o formato datetime
df2.loc[:, 'datetime'] =  pd.to_datetime(df2.datetime, format='%Y-%m-%d %H:%M:%S')

#Coloando np.nan nas medições inválidas que eu tinha marcado anteriomente como sendo de 19000
df2.loc[:, 'datetime'] = df2['datetime'].where(df2.datetime > '2000-01-01 00:00:00', np.nan)

1.04 s ± 7.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
#conferindo se ambos procedimentos geraram o mesmo resultado
teste = df.datetime.equals(df2.datetime)
print(teste)

True
